## IMPORTANT to READ and IMPLEMENT

- [ ] **Monotonicity and Convexity**: [Hofner 2012](http://benjaminhofner.de/downloa/2012/talksBiometrischesKolloquium_Hofner.pdf)
- [ ] **Automatic knot selection**: [Goepp 2018](https://arxiv.org/pdf/1808.01770.pdf) 
- [ ] **Monotonicity** for *Cubic Regression Splines*: [Wood 1994](https://epubs-siam-org.uaccess.univie.ac.at/doi/pdf/10.1137/0915069)



## Chapter 4 Some GAM theory

The problem of estimation a GAM becomes the problem of estimation *smoothing parameters* $\lambda_i$ and *model coefficients* $\beta_i$ for a *penalized likelihood maximization problem* once a *basis* for the smooth functions and a *measure of function wiggliness* has been chosen. In practice, this is solved by *P-IRLS*, while the smoothing parameters can be estimated using cross validation or related criteria. 

The methods discussed here are almost all built around penalized regression smoothers, based on splines. Some sources are

- Wahba (1980) - groudwork on penalized regression smoothers
- de Boor (1978) - introduced B-splines
- Parker and Rice (1985) - groundwork on penalized regression smoothers
- Hastie and Tibshirani (1990) - use P-Splines in the GAM context
- Marx and Eilers (1998) - improved the use for B-Splines by introducing P-Splines with a B-Spline basis

The main components of the framework for generalized additive modelling are covered in this chapter. 

#### 4.1 Smoothing bases
For simplicity of presentation, only one very simple type of penalized regression smoother was presented in Chapter 3. For practical work a variety of alternative smoothers are available, and this section introduces a useful subset of the possibilities, starting with smooths of one covariate, and then moving on to smooths of one or more covariates. Since all the smooths presented are based on splines (although the tensor product smooths need not be), the section starts by addressing the question: what’s so special about splines?

##### 4.1.1 Why splines?
Some theoretical properties that make cubic splines appealing for penalized regression, first in the concext of interpolation, and then of smoothing.

**Natural cubic splines are smoothest interpolators** \
Consider a set of points $\{ x_i, y_i: i=1,...,n\}$ where $x_i < x_{i+1}$. The *natural cubic spline*, $g(x)$, interpolating these points, is a function made up of sections of cubic polynomial, one for each $[x_i, x_{i+1}]$, which are joined together so that the whole spline is continous to second derivative, while $g(x_i) = y_i$ and $g''(x_1) = g''(x_n) = 0$.

Of all functions that are continous on $[x_1, x_n]$, have continous first derivatives and interpolate $\{x_i, y_i\}$, the natural cubic spline $g(x)$ is the smoothest in the sense of minimizing
$$
    J(f) = \int_{x_1}^{x_n} f''(x)^2 dx. 
$$
The prove is given by Green and Silverman (1994), based on the original work of Schoenberg (1964). In de Boor (1978, Cha. 5) a number of results are presented showing that cubic spline interpolation is optimal, or at least very good.

**Cubic smoothing splines** \
Usually $y_i$ is measured with noise, and it is generally more useful to smooth $\{x_i, y_i\}$ data, rather that interpolating them. Rather than setting $g(x_i) = y_i$, it might be better to treat the $g(x_i)$ as $n$ free parameters of a cubic spline, and to estimate them in order to minimize
$$
    \sum_{i=1}^n \big( y_i - g(x_i)\big)^2 + \lambda \int g''(x)^2 dx, 
$$
where $\lambda$ is a tuneable parameter controling the relative weight of the conflicting goals of matching the data and producing a smooth $g$. The result $g(x)$ is a *smoothing spline* (Reinsch, 1967). Of all functions, that are continuous on $[x_1, x_n]$ and have cont. first derivatives, $g(x)$ is the function minimizing:
$$
    \sum_{i=1}^n \big( y_i - f(x_i)\big)^2 + \lambda \int f''(x)^2 dx. \quad (4.1)
$$
Smoothing splines seem to be ideal smoothers. The only problem is that they have as many free parameters as there are data to be smoothed. This is a problem as soon as we try to deal with more covariates/dimensions.

An obvious compromise between retaining the good properties of splines, and computational efficiency, is to use penalized regression splines, as introduced in Chapter 3. At its simplest, this involves constructing a spline basis (and associated penalties) for a much smaller data-set than the one to be analyzed, and then using that basis
(plus penalties) to model the original data set. The covariate values in the smaller data set should be arranged to nicely cover the distribution of covariate values in the original data set. This penalized regression spline idea is presented in Wahba (1980) and Parker and Rice (1985), for example. In the rest of this section, some spline based
penalized regression smoothers will be presented, starting with univariate smoothers, and then moving on to smooths of several variables.

##### 4.1.2 Cubic regression splines
One approach is to parametrize the spline in terms of its values at the knots. Consider defining a cubic spline function, $f(x)$ with $k$ knots, $x_1,..., x_k$. Let $\beta_j = f(x_j)$ and $\delta_j = f''(x_j)$. Then the spline can be written as
$$
    f(x) = a_j^-(x)\beta_j + a_j^+ \beta_{j+1} + c_j^-(x) \delta_j + c_j^+(x) \delta_{j+1} \quad if \ x_j \le x \le x_{j+1} \quad (4.2)
$$
for some special form of $a$ and $c$ given in the following figure. ![](img/wood-cubic-spline-coeff.png) 
With some work, the spline can be re-written entirely in terms of $\beta$, which can then be re-written as 
$$
    f(x) = \sum_{i=1}^k b_i(x) \beta_i.
$$
Hence, given a set of $x$ values at which to evaluate the spline, it is easy to obtain a model matrix mapping $\beta$ to the evaluated spline. It is important to notice that in addition to having directly interpretable parameters, this basis does not require any re-scaling of the predictor variables before it can be used to construct a GAM. One only needs to choose the locations of the knots $x_j$. 

##### 4.1.3 Cyclic cubic regression splines
It is quite often appropriate for a model smooth function to be 'cyclic', meaning that the function has the same value and first few derviatives at its upper and lower boundaries. The penalized cubic regression spline can be modified to produce such a smooth. The spline can still be written in the form of (4.2), but we now have that $\beta_1 = \beta_k$ and $\delta_1 = \delta_k$

##### 4.1.4 P-splines
Yet another way to represent cubic splines (and indeed splines of higher or lower order), is by use of the **B-spline basis**. The B-spline basis is appealing because the basis functions are strictly local - each basis function is only non-zero over the intervals between $m + 3$ adjacent knots, where $m + 1$ is the order of the basis (e.g. $m = 2$ for a cubic spline). To define a $k$ parameter B-spline basis, we need to define $k+m+1$ knots, $x_1 < x_2 < . . . < x_{k+m+1}$, where the interval over which the spline is to be evaluated lies within $[x_{m+2}, x_k]$ (so that the first and last $m+1$ knot locations are essentially arbitrary). An $(m+1)^{th}$ order spline can be represented as
$$
    f(x) = \sum_{i=1}^k B_i^m(x) \beta_i,
$$
where the B-spline basis functions are most conveniently defined recursively as follows:
$$
    B_i^m(x) = \frac{x - x_i}{x_{i+m+1} - x_i} B_i^{m-1}(x) + \frac{x_{i+m+2} - x}{x_{i+m+2} - x_{i+1}} B_{i+1}^{m-1}(x), \quad i=1,...k
$$
and
$$
    B_i^{-1}(x) = \begin{cases}
                    1 \quad if  \ x_i \le x < x_{i+1} \\
                    0 \quad else 
                  \end{cases}
$$
(see e.g. de Boor 1978). 

B-splines were developed by de Boor as a very stable basis for large scale spline interpolation, but for most statistical work with low rank penalized regression spline, you would have to be using very poor numerical methods before the enhanced stability of the basis became noticable. The real statistical interest in B-splines has resulted from the work of Eilers and Marx (1996) in using them to develop what they term *P-splines*.

P-splines are low rank smoothers using a B-spline basis, usually defined on evenly spaced knots, and a difference penalty applied directly to the parameters $\beta_i$ to control function wiggliness. If the squared difference between adjacent $\beta_i$ values are penalized then the penalty would be
$$
    \mathcal P = \sum_{i=1}^{k-1} (\beta_{i+1} - \beta_i)^2 = \beta_1^2 - 2 \beta_1 \beta_2 + 2 \beta_2^2 - 2\beta_2 \beta_3 + ... + \beta_k^2,
$$
and it is straightforward to see that this can be written as
$$
    \mathcal P = \beta^T \begin{bmatrix} 1 & -1 &  0& .& . \\
                                        -1 &  2 & -1& .& . \\
                                         0 & -1 &  2& .& . \\
                                         . &  . &  .& .& . \\
                                         . &  . &  .& .& .
                         \end{bmatrix} \beta. 
$$
Higher order penalties are also possible.

P-splines are extremely easy to set up and use, and allow a good deal of flexibility, in that any order of penalty can be combined with any order of B-spline basis. Their disadvantage is that the simplicity is somewhat diminished if uneven knot spacing is required and that the penalties are less easy to interpret in terms of properties of fitted smooth than the more usual spline penalites. 

* [ ] Exercise 7
* [ ] Exercise 9 

##### 4.1.5 Thin plate regression splines
The bases covered so far are each useful in practice, but are open to some criticism:

1. necessary to choose knot locations, therefor extra degree of subjectivity
2. only useful representing smooths of one predictor variable
3. not clear to what extend this bases are better or worse than any other basis that might be used

**Thin plate splines** \
They were introduced by Duchon (1977). Very elegant and general solution to the problem of estimating a smooth function of multiple predictor variables, from noisy observations of the function. The problem is to estimate a smooth function $g(x)$, from $n$ observations $(x_i, y_i)$ such that
$$
    y_i = g(x_i) + \epsilon_i
$$ 
where $\epsilon_i$ is a random error term and $x \in \mathbb R^d$ ($d \le n$).

Thin plate spline smoothing estimates $g$ by finding the function $\hat f$ minimizing:
$$
    \lVert y - f \rVert^2 + \lambda J_{md}(f) \quad (4.4)
$$
where $y$ is the vector of data and $f = (f(x_1), f(x_2), ..., f(x_n))^T$. $J_{md}(f)$ is a penalty function measuring the 'wiggliness' of $f$ and $\lambda$ is a smoothing parameter. $J_{md}(f)$ is defined as
$$
    J_{md}(f) = \int ... \int_{\mathbb R} \sum_{\nu_1+...+\nu_d=m} \frac{m!}{\nu_1!...\nu_d!} \big( \frac{\partial^m f} {\partial x_1^{\nu_1}... \partial x_d^{\nu_d}}\big)^2 dx_1 ... dx_2. \quad (4.5)
$$
The penalty function looks somewhat intimidating, but an example will help. In the case of a smooth of two predictors with wiggliness measured using second derivatives, we have
$$
    J_{22} = \int \int \big(\frac{\partial^2 f}{\partial x_1^2}\big)^2 +
                       \big(\frac{\partial^2 f}{\partial x_1 x_2}\big)^2 + 
                       \big(\frac{\partial^2 f}{\partial x_2^2}\big)^2 dx_1 dx_2.
$$
Further progress is only possible if $m$ is choosen so that $2m > d$ (for 'visually smooth' results one needs $2m > d+1$. Subject to the first of these restrictions, it can be shown that the function minimizing (4.4) has the form
$$
    \hat f(x) = \sum_{i=1}^n \delta_i \eta_{md}(\lVert x - x_i \rVert) + \sum_{j=1}^M \alpha_j \phi_j(X), \quad (4.6)
$$
where $\alpha$ and $\delta$ are the coefficients to be estimated, $\delta$ being subject to the linear constraint that $T^T \delta = 0$ where $T_{ij} = \phi_j(x_i)$. The $M = \binom{m+d-1}{k}$ functions $\phi_i$ are linearly independent polynomials spanning the space of polynomials in $\mathbb R^d$ of degree less than $m$. For example, for $m = d = 2$ these functions are $\phi_1(x) = 1, \phi_2(x) = x_1$ and $\phi_3(x) = x_2$. The remaining basis functions used in (4.6) are defined as 
$$
    \eta_{md}(r) = \begin{cases}
                            \frac{(-1)^{m+1+d/2}}{2^{2m-1} \pi^{d/2} (m-1)!(m-d/2)!} r^{2m-d} \log (r) \quad if \ d \ even \\
                            \frac{\Gamma(d/2 - m)}{2^{2m} \pi^{d/2} (m-1)!} r^{2m-d} \quad \quad \quad if \ d \ even
                    \end{cases}
$$

Now defining the matrix $E$ by $E_{ij} = \eta_{md}(\lVert x_i - x_j \rVert)$, the thin plate spline fitting problem becomes, 
$$
    \min_{\alpha, \delta} \lVert y - E \delta - T \alpha \rVert^2 +  \lambda \delta^T E \delta \ s.t. \ T^T \delta = 0, \quad (4.7)
$$
The thin plate spline $\hat f$ is something of an ideal smoother: it has been constructed by defining

- what is meant by smoothness
- how much weight to give the conflicting goal of matching the data and making $\hat f$ smooth
- finding the function that best satisfies the resulting smoothing objective. 

One did not need to choose knot positions or select basis functions. In addition, TPS can deal with an arbitrary number of predictors. Here also lies the problem of TPS: the computational cost is proportional to the cube of the number of parameters (which is the number of data $n$). Given that the effective degrees of freedom estimated for a model term is usually a small proportion of $n$, the question arises whether a low rank approximation could be produced which is as close as possible to the TPS, without incurring the high computational cost.

**Thin plate regression splines** \
Are based on the idea of truncating the space of wiggly componets of the TPS (with parameters $\delta$), while leaving the components of 'zero wiggliness' unchanged (with parameters $\alpha$). This is done by an eigen-decomposition of the matrix $E$ into $E = UDU^T$. Now only take the first $k$ columns of $U$ and denote them $U_k$. $D_K$ denotes the top right $k \times k$ submatrix of $D$. Restricting $\delta$ to the columns space of $U_k$, by writing $\delta = U_k \delta_k$, means that (4.7) becomes 
$$
    \min_{\alpha, \delta_k} \lVert y - U_k D_k \delta_k - T \alpha \rVert^2 +  \lambda \delta_k^T D_k \delta_k \ s.t. \ T^U_k \delta_k = 0.
$$
The constraints can be absorbed in the usual manner, described in Section 1.8.1 (General linear constraints) and A.6 (QR-decomposition).

First we find any orthogonal column basis $Z_k$, such that $T^T U_k Z_k = 0$ (e.g. by $QR$ decomposition of $U_k^T T$: the final $M$ columns of the orthogonal factor give a $Z_k$). Restricting the $\delta_k$ to this space ( $\delta_k = Z_k \tilde \delta$), yields the unconstrained problem that must be solved to fit the rank $k$ approximation to the smoothing spline:
$$
    \min_{\alpha, \tilde \delta} \lVert y - U_k D_k Z_k \tilde \delta - T \alpha \rVert^2 +  \lambda \tilde \delta^T Z_k^T D_k Z_k \tilde \delta 
$$
This has a computational cost of $\mathcal O(k^3)$. Having the fitted model, evaluation of the spline at any point is easy: just evaluate $\delta = U_K Z_K \tilde \delta$ and use (4.6). The eigen-decomposition can be found by Lanczos iteration ($\mathcal O(n^2k)$ )for large systems and be full $QR$ ($\mathcal O(n^3)$) for small systems. 

**Properties of TPRS** \
TPRS properties are:

- avoid knot placement
- relatively cheap to compute
- can use any number of predictor variables
- [ ] what about optimality ? 

**Knot based approximation** \
If knot locations $\{x_i^*: i=1...k\}$ are chosen, then the spline can be approximated by
$$
    \hat f(x) = \sum_{i=1}^k \delta_i \eta_{md}(\lVert x - x_i^*\rVert) + \sum_{j=1}^M \alpha_j \phi_j(x), \quad (4.8)
$$
where $\delta$ and $\alpha$ are estimated by minimizing
$$
    \lVert y - X \beta \rVert^2 + \lambda \beta^T S \beta, \quad \ s.t. \ C\beta = 0
$$
w.r.t $\beta^T = (\delta^T, \alpha^T)$. $X$ is an $n \times k + M$ matrix such that
$$
    X_{ij} = \begin{cases}
                \eta_{md} (\lVert x_i - x_j^* \rVert ) \quad j=1, ..., k \\
                \phi_{j-k}(x_i) \quad j=k+1, ..., k + M,
             \end{cases}
$$

$S$ is a $(k + M) \times (k + M)$ matrix with zeros everywhere except in its upper left $k \times k$ block where $S_{ij} = \eta_{md}(\lVert x_i^* - x_j^* \rVert)$. Finally, $C$ is an $M \times (k + M)$ matrix such that 
$$
    C_{ij} = \begin{cases}
                \phi_i(x_j^*) \quad j=1, ..., k \\
                0 \quad j = k+1, ..., k+M.
             \end{cases}
$$
This approximation goes back at least to Wahba (1980). Some care is required to choose the knot locations. For more dimensions it is often difficult. One possibility is to take a random sample of observed predictor variable combinations, another is to take a 'spatially stratified' sample of the predictor variable combinations. Even spaceing is sometimes appropriate, or more sophisticated space filling schemes can be used: Ruppert et al. (2003) provided a useful discussion of the alternatives. 

- [ ] Section 1.8.1: Constraint - absorbtion

##### 4.1.6 Shrinkage smoothers

A disadvantage of the smooths discussed so far, is that no matter how large their associated smoothing parameter becomes, the smooth is never completely eliminated in the sense of having all its parameters estimated to be zero. On the contrary, some functions are treated as completely smooth by the penalty, and hence functions of this class are always completely un-penalized. From the point of view of *model selection* with GAMs it would be more convenient if smooths could be zeroed by adjustment of smoothing parameters. 

A fairly crude alternative, is simply to add a small multiple of the identity matrix to the penalty matrix of the smooth, i.e. 
$$
    S \rightarrow S + \epsilon I
$$
so that the penalty will now shrink all parameters to zero if its associated smoothing parameter is large enough. If $\epsilon$ is small enough, the identity part of the penalty
will have almost no impact when a function is ‘wiggly’: only once it becomes close
to ‘completely smooth’ will the identity component start to become important, and
really start shrinking the parameters towards zero.

##### 4.1.7 Choosing the basis dimension

When using penalized regression splines the modeller chooses the basis dimension as part of the model building process. Typically, this substantially reduces the computational burden of modelling, relative to full spline methods, and recognizes the fact that, usually, something is seriously wrong if a statistical model really requires as many coefficients as there are data. 

The main challenge introduced, by this low rank approach, is that a basis dimension has to be chosen. In the context of spline smoothing, Kim and Gu (2004) showed that the basis size should scale as $n^{2/9}$, where $n$ is the number of data.

In practice, then, choice of basis dimension is something that probably has to remain a part of model specification. However, it is important to note that the exact size of basis dimension is really not that critical. The basis dimension is only setting an upper bound on the flexibility of a term: it is the smoothing parameter that controls the actual effective degrees of freedom. Hence the model fit is usually rather insensitive to the basis dimension, provided that it is not set restrictively low for the application concerned. 

The only caveat to this point is the slightly subtle one, that a function space with basis dimension 20 will contain a larger space of functions with EDF 5 than will a function space of dimension 10 (the numbers being arbitrary): it is this fact that causes model fit to retain some sensitivity to basis dimension, even if the appropriate EDF for a term is well below the basis dimension.In practice, the modeller needs to decide roughly how large a basis dimension is fairly certain to provide adequate flexibility, in any particular application, and use
that.

##### 4.1.8 Tensor product smooths

A major feature of the TPS/TPRS approach is the isotropy of the wiggliness penalty: wiggliness in all directions is treated equally, with the fitted spline entirely invariant to rotation of the co-ordinate system for the predictor variables. This isotropy is often considered to be desirable (especially as a smooth function of geographic co-ordinates), but comes with some disadvantages, e.g. the difficulty of knowing how to scale predictors relative to each other, when they are measured in fundamentally different units. One pragmatic approach is to scale all predictors into the unit square. A more satisfactory approach uses *tensor product smooths*. 

**Tensor product bases** \
The basic approach of this section is to start from smooths of single covariates, represented using any basis with associated quadratic penalty measuring 'wiggliness' of the smooth. From these 'marginal smooths' a 'tensor product' construction is used to build up smooths of several variables. See de Boor (1978) for an important early reference on tensor product spline bases.

As example, a smooth function of 3 covariates $x, z$ and $v$ is constructed (in general this can be done with an arbitrary number of covariates). The process starts by assuming that we have low rank bases available, representing smooth functions $f_x$, $f_z$ and $f_v$. That is
$$
    f_x(x) = \sum_{i=1}^I \alpha_i a_i(x), \quad 
    f_z(z) = \sum_{l=1}^L \delta_l d_i(z), \quad
    f_v(v) = \sum_{k=1}^K \beta_k b_i(x)
$$
where $\alpha_i$, $\delta_l$ and $\beta_k$ are parameters and $a_i(x)$, $d_l(z)$ and $b_k(v)$ are known basis functions.  

What is required for $f_x$ to vary smoothly with $z$? This can be achieved by allowing its parameters $\alpha_i$ to vary smoothly with $z$. Using the already known basis we could write
$$
    \alpha_i(z) = \sum_{l=1}^L \delta_{ij} d_l(z)
$$
which immediately gives
$$
    f_{xz}(x, z) = \sum_{i=1}^I \sum_{l=1}^L \delta_{ij} d_l(z) a_i(x).
$$
In the same way, a smooth function of any number of covariates can be construted. In our example, this leads then to
$$
    f_{xzv}(x,z,v) = \sum_{i=1}^I \sum_{l=1}^L \sum_{k=1}^K \beta_{ilk} b_k(v) d_l(z) a_i(x) 
$$
The model matrix $X$ can be constructed by using the Kronecker product, given an appropriate ordering of the $\beta_{ijl}$ into a vector $\beta$. The $i^{th}$ row of $X$ is simply
$$
    X_i = X_{xi} \otimes X_{zi} \otimes X_{vi}.
$$
From this follows:

- the construction can be done for an arbitrary number of covariates
- the results are independent of the order in which the covariates are treated
- the covariates can themselves be vector covariates

**Tensor product penalties** \
Suppose each marginal smooth has an associated functional measuring function wiggliness which can be expressed as a quadratic form in the marginal parameters, like
$$
    J_x(f_x) = \alpha^T S_x \alpha, \quad
    J_z(f_z) = \delta^T S_z \delta, \quad
    J_v(f_v) = \beta^T S_v \beta.
$$
The $S_*$ matrices contain known coefficients, and the $\alpha$, $\delta$ and $\beta$ are vectors of coefficients of the marginal smooths. Each marginal penalty is the associated with a smoothing parameter $\lambda_*$. Hence, if the marginal penalties are easily interpretable, in terms of function shape (e.g. the cibuc spline penalty), then so is the induced penalty. As an example, if cubic spline penalties were used as the marginal penalties, then
$$
    J(f) = \int_{x,z,v} \lambda_x \big( \frac{\partial^2 f}{\partial x^2}\big) + 
                        \lambda_z \big( \frac{\partial^2 f}{\partial z^2}\big) +
                        \lambda_v \big( \frac{\partial^2 f}{\partial v^2}\big) dx dz dv.
$$
The integral can be performed numerically, and it is clear that the same approach can be applied to all components of the penalty. However, a simple reparameterization (p. 162f) can be used to provide an approximation to the terms in the penalty, which performs well in practice, and avoids the need for explicit numerical integration.

In [278]:
import numpy as np
from numpy.linalg import lstsq
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

init_notebook_mode(connected=True) 
np.random.seed(42)

class BSpline:
    
    def __init__(self, order="cubic", X=None):
        
        if order is "cubic":
            self.order = 2
        else:
            self.order = 0
        
        if X is None:
            print("No data inserted!")
        else:
            self.X = x
    def recursiveDef(self, xk, i, m=2):
        """
        compute the i-th b-spline basis function of order m at the values given in x, given
        knot locations in xk
        Parameter:
        ---------------------
        x   : array,     of data, predictor -- atm. one dim.
        xk  : array,     of knot locations
        i   : int,       idx of the b-spline basis function to compute
        m   : int,       order of the spline, default is 2 (cubic)
        """
        if m == -1:
            return (x >= k[i]) & (x < k[i+1]).astype(int)
        else:
            z0 = (x - k[i]) / (k[i+m+1])
            z1 = (k[i+m+2] - x) / (k[i+m+2] - k[i+1])
            return z0*self.recursiveDef(self.X, xk, i, m-1) + \
                   z1*self.recursiveDef(self.X, xk, i+1, m-1)
    
    def calcPSplinePenaltyMatrix(self, rank):
        """
        compute the pSpline penalty matrix of 2nd order according to p.150 Wood 2006
        """
        self.pSplinePenaltyMatrixRank = rank
        a = np.eye(rank)
        P = np.diff(a, axis=0)
        S = np.dot(P.T, P)
        self.pSplinePenaltyMatrix
        return S

    
    def modelMatrixBSpline(self, k=0, m=2):
        """ 
        set up model matrix for the B-spline basis
        one needs k + m + 1 knots for a spline basis of order m with k parameters
        k - number of parameters (== number of B-splines)
        """
        x = self.X
        n = len(x) # n = number of data
        assert (k > 0), "number of B-splines k need to be specified and larger than 0"
        xmin, xmax = np.min(x), np.max(x)
        xk = np.quantile(a=x, q=np.linspace(0,1,k))
        # Change the next 2 lines
        xk = np.insert(xk, 0, np.linspace(xmin-1, xmin, m+1)[:-1]) # leave the last inserted point out
        xk = np.insert(xk, -1, np.linspace(xmax, xmax+1, m+1))
        X = np.zeros(shape=(n, k))
        for i in range(k):
            X[:,i] = bSpline(x=x, k=xk, i=i, m=m)
        self.modelMatrix = X
        return X

    def plotModelMatrix(self, log=False):
        if self.X is not None:
            X = self.modelMatrix
            if log:
                X = np.log(X + np.ones(X.shape)*1e-8)
            print("Shape of X:", X.shape)
            fig = px.imshow(X)
        else:
            print("ModelMatrix is not available!")
        return fig
    
    def fitModel(self, y, k):
        """
        fit the linear model with the basis given by the B-spline basis of k parameters using
        linear least squares with y = X.T beta
        Parameters:
        ---------------
        y     : array     - target data
        k     : int       - number of parameters of the fit
        """
        self.modelMatrix = self.modelMatrixBSpline(k=k)
        self.y = y
        
        fit = lstsq(a=self.modelMatrix, b=y)
        return fit
    
    def plotModel(self, fit):
        
        x = self.X
        y = self.y
        fittedValues = np.dot(self.modelMatrix, fit[0])
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, name="data", mode="markers",         
                                 marker=dict(color='LightSkyBlue', size=4, line=dict(color='#17becf',width=2))))
        fig.add_trace(go.Scatter(x=x, y=fittedValues, name="model", mode="markers+lines",
                                marker=dict(color='#d62728', size=4, line=dict(color='#d62728',width=2))))
        fig.show()
        return fig
        
        
def bSpline(x, k, i, m=2):
    """ evaluate i-th b-spline basis function of order m at the values in x, given knot 
        loactions in k
    """
    if m==-1:
        # return 1 if x is in {k[i], k[i+1]}, otherwise 0
        return (x >= k[i]) & (x < k[i+1]).astype(int)
    else:
        #print("m = ", m, "\t i = ", i)
        z0 = (x - k[i]) / (k[i+m+1] - k[i])
        z1 = (k[i+m+2] - x) / (k[i+m+2] - k[i+1])
        return z0*bSpline(x, k, i, m-1) + z1*bSpline(x, k, i+1, m-1)

def addVertLinePlotly(fig, x0=0, y0=0, y1=1):
    """ plots a vertical line to the given figure at position x"""
    fig.add_shape(
        dict(
            type="line", 
            x0=x0, x1=x0, 
            y0=y0, y1=1.2*y1, 
            line=dict(color="LightSeaGreen", width=1)
        )
    )
    return

def pSplinePenalty(k):
    """ 
    compute the P-spline second order difference penalty matrix of  dimension 
    k according to p.150 Wood 2006 
    """
    assert (type(k) is int), "Type of input k is not int"
    a =np.eye(k)
    P = np.diff(a, axis=0)
    S = np.dot(P.T,P)
    return S

def monotonicIncreasingPenaltyMatrix(k):
    """
    compute the penalty matrix for monotonic increasing b-splines accoring to De Boor (1978) or 
    https://www-tandfonline-com.uaccess.univie.ac.at/doi/full/10.1080/00031305.2014.969445?src=recsys&journalCode=utas20
    k = nr of parameters
    returns the matrix A \in R^{k-1 x k}
    """
    assert (type(k) is int), "Type of input k is not int"
    b = np.eye(k, dtype=np.int)
    A = np.diff(b, axis=0)
    return A

def peakPenaltyMatrix(k, peakLoc, xk):
    """
    create a penalty matrix as mixture of monotonic inreasing and decreasing penalty matrix, where 
    the mixture point is given py the peakLoc
    Parameter:
    -------------
    k       : int     - size of the penalty matrix
    peakLoc : float   - guess for the peak point
    xk      : array   - array of knot locations
    """
    idx_before_peak = int(np.argwhere(xk < peakLoc)[-1])
    #print(idx_before_peak)
    increasing = monotonicIncreasingPenaltyMatrix(idx_before_peak)
    decreasing = -1 * monotonicIncreasingPenaltyMatrix(k - idx_before_peak)
    #print(increasing.shape)
    #print(decreasing.shape)
    
    P = np.zeros((k-1,k))
    P[:idx_before_peak-1, :idx_before_peak] = increasing
    P[idx_before_peak:, idx_before_peak:] = decreasing
    #print("shape P: ", P.shape)
    return P
    
    
def modelMatrixBSpline(x, k=0, m=2):
    """ 
    set up model matrix for the B-spline basis
    one needs k + m + 1 knots for a spline basis of order m with k parameters
    k - number of parameters (== number of B-splines)
    """
    n = len(x) # n = number of data
    assert (k > 0), "number of B-splines k need to be specified and larger than 0"
    xmin, xmax = np.min(x), np.max(x)
    xk = np.quantile(a=x, q=np.linspace(0,1,k))
    dx = xk[2] - xk[1]
    xk = np.insert(xk, 0, np.arange(xmin-(m+1)*dx, xmin, dx))
    xk = np.insert(xk, -1, np.arange(xmax, xmax+(m+1)*dx, dx)[1:])
    X = np.zeros(shape=(n, k))
    for i in range(k):
        X[:,i] = bSpline(x=x, k=xk, i=i, m=m)
    return X, xk

def f1(x): return np.tanh(-(x-5)) + np.exp(-(x)**2) + np.random.randn(len(x))*0.1

def f2(x, a=1, b=1): return a / (1 + (b*x)**2) + np.random.randn(len(x))*0.1

def peak_fit(x, y, k, lam_w=0, lam_m=0, lam_p=0, plot_=0, x_test=None):
    """
    penalizes spline regression with a wiggliness penalty and a monotonic or 
    (exlusive or) peak penalty
    """
    X, xk = modelMatrixBSpline(x, k=k)
    smoothPenMat = pSplinePenalty(k)
    monPenMat = monotonicIncreasingPenaltyMatrix(k)
    peakPenMat = peakPenaltyMatrix(k, peakLoc=x[np.argmax(y)], xk=xk)
    
    X_aug, y_aug = X, y
    # add smoothnes penalty matrix
    if lam_w:
        y_aug = np.concatenate((y, np.zeros(shape=smoothPenMat.shape[0],)))
        X_aug = np.concatenate((X, lam_w*smoothPenMat))
    
    # add monotonic penalty matrix
    if lam_m:
        y_aug = np.concatenate((y_aug, np.zeros(shape=monPenMat.shape[0],)))
        X_aug = np.concatenate((X_aug, lam_m*monPenMat))
    
    # add peak penalty matrix
    if lam_p:
        y_aug = np.concatenate((y_aug, np.zeros(shape=peakPenMat.shape[0],)))
        X_aug = np.concatenate((X_aug, lam_p*peakPenMat))


    fit = lstsq(a=X_aug, b=y_aug)
    y_hat = X.dot(fit[0])
    if plot_:
        fig = px.scatter(x=x, y=y)
        fig.add_trace(go.Scatter(x=x, y=y_hat, name="fit", mode="markers"))
        fig.show()
        
    y_pred = None
    if x_test is not None:
        X_t, xk_t = modelMatrixBSpline(x_test, k=k)
        y_pred = X_t.dot(fit[0])
    return y_hat, y_pred

In [279]:
# test the peak fit abiltiy     
x = np.linspace(-3, 10, 1000)
y = ff(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
y_train.shape

(700,)

In [280]:
k = 75

mse_list = list()
fit_list = list()
lam_p = 0.01
for i in range(10):
    
    fit = peak_fit(x_train, y_train, k=k, lam_w=1, lam_p=lam_p, plot_=0,x_test=x_test)
    mse_list.append(np.round(mse(y_test, fit[1]), 6))
    fit_list.append(fit[0])
    lam_p *= 2.5
    print("Lambda peak: ", lam_p)

    
print("Mean Squared Errors: ", mse_list)
px.scatter(x=x_train, y=fit_list[np.argmin(mse_list)])

C:\Users\jakob\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:237: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

C:\Users\jakob\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:237: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

C:\Users\jakob\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:237: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence

Lambda peak:  0.025
Lambda peak:  0.0625
Lambda peak:  0.15625
Lambda peak:  0.390625
Lambda peak:  0.9765625
Lambda peak:  2.44140625


C:\Users\jakob\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:237: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

C:\Users\jakob\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:237: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

C:\Users\jakob\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:237: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence

Lambda peak:  6.103515625
Lambda peak:  15.2587890625
Lambda peak:  38.14697265625
Lambda peak:  95.367431640625
Mean Squared Errors:  [0.012387, 0.012387, 0.012387, 0.012384, 0.01237, 0.012295, 0.012039, 0.012397, 0.020365, 0.04218]


In [296]:
y = ff(x, a=2)
k=75
lam = 0.01
X, xk = modelMatrixBSpline(x, k=k)
smoothPenMat = pSplinePenalty(k)
monPenMat = monotonicIncreasingPenaltyMatrix(k)
peakPenMat = peakPenaltyMatrix(k, peakLoc=0, xk=xk)

# base case
X_aug, y_aug = X, y

## add smoothnes penalty matrix
y_aug = np.concatenate((y, np.zeros(shape=smoothPenMat.shape[0],)))
X_aug = np.concatenate((X, lam*smoothPenMat))
#
## add monotonic penalty matrix
#y_aug = np.concatenate((y_aug, np.zeros(shape=monPenMat.shape[0],)))
#X_aug = np.concatenate((X_aug, 50*lam*monPenMat))
#
# add peak penalty matrix
y_aug = np.concatenate((y_aug, np.zeros(shape=peakPenMat.shape[0],)))
X_aug = np.concatenate((X_aug, 
                        400*lam*peakPenMat))

print("Shape X_aug: ", X_aug.shape)
print("Shape y_aug: ", y_aug.shape)
fit = lstsq(a=X_aug, b=y_aug)
fit[0]
y_hat = X.dot(fit[0])
fig = go.Figure()
#fig = px.scatter(x=x, y=y)
#fig.add_trace(go.Scatter(x=x, y=y_hat, name="fit"))
fig.add_trace(go.Scatter(x=x[:-1], y=np.diff(y_hat), mode="markers"))
fig.show()

Shape X_aug:  (399, 75)
Shape y_aug:  (399,)


C:\Users\jakob\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:27: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



In [49]:
# quadratic prgram: xT Q x - cT x with Q = XT X + lambda S and c = (-2 XT b)T

Q = np.dot(X_aug.T, X_aug)
c = -2 * np.dot(X_aug.T, y_aug).T
G = 1*monPenMat
h = np.zeros(shape=(G.shape[0],))
A = np.zeros((k,))
b = np.zeros((1,))
print("Shape Q: ", Q.shape)
print("Shape c: ", c.shape)
print("Shape G: ", G.shape)
print("Shape h: ", h.shape)
from qpsolvers import solve_qp
s = solve_qp(P=Q,q=c, G=G, h=h, A=A, b=b)
len(s)
fig = px.scatter(x=x, y=y)
fig.add_trace(go.Scatter(x=x, y=np.dot(X, s), name="fit"))
fig.show()

Shape Q:  (10, 10)
Shape c:  (10,)
Shape G:  (9, 10)
Shape h:  (9,)


In [288]:
np.arange(xmax, xmax+(m+2)*dx, dx)

array([5.        , 5.33333333, 5.66666667, 6.        ])

In [293]:
# plot the B-Spline basis functions
x = np.linspace(-2,5,250)
k, m = 10, 2
xmin, xmax = np.min(x), np.max(x)
xk = np.quantile(a=x, q=np.linspace(0,1,k))
dx = xk[2] - xk[1]
print(dx)
xk = np.insert(xk, 0, np.arange(xmin-(m+1)*dx, xmin, dx))
xk = np.append(xk, np.arange(xmax, xmax+(m+1)*dx, dx)[1:])
print(xk)
X = np.zeros(shape=(len(x), k))

fig = go.Figure()

for i in range(k):
    b = bSpline(x, xk, i, m=2)
    X[:,i] = b
    fig.add_trace(go.Scatter(x=x, y=b))

for i in range(len(xk)):
    addVertLinePlotly(fig=fig, x0=xk[i])
fig.show()


0.7777777777777777
[-4.33333333 -3.55555556 -2.77777778 -2.         -1.22222222 -0.44444444
  0.33333333  1.11111111  1.88888889  2.66666667  3.44444444  4.22222222
  5.          5.77777778  6.55555556]


In [283]:
from numpy.linalg import lstsq

def f(x): return 2*np.tanh(x) + np.random.randn(len(x)) * 0.1

x = np.linspace(-2,5,500)
y = f(x)
k = 25
lam = 1e-8

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode="markers", name="data"))

for i in range(25):
    modMatrix = modelMatrixBSpline(x=x, k=k, m=2)
    penMatrix = pSplinePenalty(k=k)
    X_aug = np.concatenate((modMatrix, lam*penMatrix))
    y_aug = np.concatenate((y, np.zeros(penMatrix.shape[1])))
    mod = lstsq(a=X_aug, b=y_aug)
    if not i % 5: fig.add_trace(go.Scatter(x=x, y=np.dot(modMatrix, mod[0]), name="lambda={}".format(np.round(float(lam), 5))))
    lam = lam * 5
fig.show()


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)